In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
from collections import defaultdict
from glob import glob
from scipy.stats.mstats import gmean
from scipy.optimize import curve_fit
from scipy.stats import norm,linregress,ranksums,ttest_ind,ttest_ind_from_stats
from sklearn.metrics import precision_recall_curve,roc_curve,auc
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm,CenteredNorm
from matplotlib import cm
from matplotlib.ticker import LogLocator, LogFormatterSciNotation
from matplotlib_venn import venn2,venn3
plt.rcParams['svg.fonttype']='none'
from itertools import combinations as combs
import pickle
INPUT_DIR = '../../data/Experimental_Results'
OUTPUT_DIR = '../../output/Experimental_Results'

### Load viability data and metadata

`reduced_dosage_d`: a Python dictionary with keys corresponding to drugs and values indicating the number of nanoliters added to the plate. 40nL = 10uM.

In [ ]:
reduced_dosage_d = {'doxorubicin':2.5,'torin 2':2.5,'gsk1120212':20,
'mycophenolic acid':20,'ciclopirox':20,'daunorubicin':2.5,
'digoxin':2.5,'digitoxin':2.5,'topotecan':2.5,
'melphalan':20,'zm447439':20,'anisomycin':20,
'sgc-cbp30':20,'calcipotriol':20,'lomefloxacin':20}

`sel_df_comb`: a pandas DataFrame containing the viability data from each well for each cell line, plate, and replicate.

Rows are labeled by plate wells and columns are labeled by a tuple of (`Cell_Line`, `Plate_Layout`, `Replicate`).

Cell Line can be one of `CH`:CH-157MN, `IO`:IOMM-LEE, `43`:GBM43, or `LN`:LN-229.

Plate Layout can be `A`:alpha or `B`:beta, specifying the layout detailed in Supplementary Table S11.

In [ ]:
sel_df_comb = pd.read_csv('%s/viability_data.csv' % INPUT_DIR,index_col=0,header=[0,1,2])

`drug2moa`: a Python dictionary with keys corresponding to drug names and values corresponding to mechanism(s) of action.

In [ ]:
def read_drug2moa(fn):
    drug2moa = {}
    with open(fn,'r') as fh:
        for ln in fh:
            if ln.startswith('##'):
                continue
            k,vv = ln.split('\t')
            if ',' in vv:
                drug2moa[k]=vv.rstrip().split(',')
            else:
                drug2moa[k]=vv.rstrip()
    return drug2moa
drug2moa =read_drug2moa('%s/drug2moa.txt' % INPUT_DIR)

`test_grps`: pandas DataFrame that contains the plate layout for the experiment, including the drug group and prediction rank

`transfer_file`: spreadsheet file used to operate the Echo robot that dispenses the drugs. It includes a mapping of drugs names to CAS numbers and NU-HTAL ids in addition to mapping the wells.

`alpha_plate`: drugs that appear on the "alpha" plate layout

`beta_plate`: drugs that appear on the "beta" plate layout

In [ ]:

test_grps = pd.read_excel('%s/TableS11_PlateSchematic.xlsx' % INPUT_DIR,sheet_name='Drug Groupings',engine='openpyxl',index_col=[0,1])
transfer_file = pd.read_excel('%s/transfer_file.xlsx' % INPUT_DIR,engine='openpyxl',index_col=0,header=0)
alpha_plate = transfer_file[transfer_file['Destination Plate']=='IOMM-LEE_alpha_1']
beta_plate = transfer_file[transfer_file['Destination Plate']=='IOMM-LEE_beta_1']

GB_alpha = alpha_plate.groupby('Destination Well')
GB_beta = beta_plate.groupby('Destination Well')



`index_dict`: dictionary mapping different groups to top selected drugs (the top overall selected drugs as well as those that are specific to given methods)

The following are dictionaries mapping drug pairs of specified categories to their wells & total volume of the drugs.
1. `sel_inds_d`:  top selected drug pairs 
2. `sel_inds_singles`: single-drug treatments of the top selected drugs 
3. `noncanc_alpha_d`: non-cancer drugs and drug pairs that are on the 'alpha' layout plate
4. `noncanc_beta_d`:  non-cancer drugs and drug pairs that are on the 'beta' layout plate
5. `ld_inds_d`: low-dosage drugs and drug pairs
6. `antisel_well_d`: anti-selected drug pairs
7. `nonsel_inds_d`: randomly-selected drug pairs

In [ ]:
index_dict = pd.read_pickle('%s/index_dict.pkl' % INPUT_DIR)
sel_inds_d = pd.read_pickle('%s/sel_inds_d.pkl' % INPUT_DIR)
sel_inds_singles = pd.read_pickle('%s/sel_inds_singles.pkl' % INPUT_DIR)
noncanc_alpha_d = pd.read_pickle('%s/noncanc_alpha_d.pkl' % INPUT_DIR)
noncanc_beta_d = pd.read_pickle('%s/noncanc_beta_d.pkl' % INPUT_DIR)
ld_inds_d2 = pd.read_pickle('%s/ld_inds_d.pkl' % INPUT_DIR)
ld_inds_d = {}
for k,v in ld_inds_d2.items():
    if isinstance(k,tuple):
        ld_inds_d[tuple(list(sorted(k)))]=v
    else:
        ld_inds_d[k]=v
antisel_well_d = pd.read_pickle('%s/antisel_well_d.pkl' % INPUT_DIR)
nonsel_inds_d2 = pd.read_pickle('%s/nonsel_inds_d.pkl' % INPUT_DIR)
nonsel_inds_d = {}
for k,v in nonsel_inds_d2.items():
    if isinstance(k,tuple):
        nonsel_inds_d[tuple(list(sorted(k)))]=v
    else:
        nonsel_inds_d[k]=v


dictionaries mapping the cell line & drug to
1. `cl_drg_data_d`: a list of viability data
2. `cl_drg_inds_d`: a list of tuples of the form (plate_well,(layout,replicate))

In [ ]:
## convert sel_df_comb into dictionaries that make the 
## viability data accessible by drug pair
cl2abbr = {'CH-157MN':'CH','GBM12':'12','GBM43':'43','IOMM-LEE':'Io','LN-229':'LN'}
cl_drg_data_d = {}
cl_drg_inds_d = {}
for CL,clabbr in cl2abbr.items():
    if CL=='GBM12':
        continue
    cl_data = sel_df_comb.xs(clabbr,level=0,axis=1)
    ## need to get A/B / 1/2/3
    drg_data_d = defaultdict(list)
    drg_inds_d = defaultdict(list)
    for aorb,cols in cl_data.groupby(level=0,axis=1):
        cols2 = cols.columns.droplevel(0)
        if aorb == 'A':
            pver = 'alpha'
            GB =  GB_alpha
            drg_well_d = {}
            drg_well_d.update(dict([(k,[v]) for k,v in sel_inds_d.items()]))
            drg_well_d.update(dict([(k,[v]) for k,v in sel_inds_singles.items()]))
            drg_well_d.update(dict([(k,[v]) for k,v in noncanc_alpha_d.items()]))
        elif aorb == 'B':
            pver = 'beta'
            GB =  GB_beta
            drg_well_d = {}
            drg_well_d.update(dict([(k,[v]) for k,v in noncanc_beta_d.items()]))
            drg_well_d.update(dict([(k,[v]) for k,v in ld_inds_d.items()]))
            drg_well_d.update(dict([(k,[v]) for k,v in nonsel_inds_d.items()]))
            drg_well_d.update(dict([(k,[v]) for k,v in antisel_well_d.items()]))
        ## correct the drug_well_d
        rep_well_d = dict([(ii,drg_well_d.copy()) for ii in range(1,4)])        
        ## using rep_well_d, iterate over all drugs in drg_well_d and assign (well,aorb,rep) to each available drug pair
        for k in drg_well_d.keys():
            if isinstance(k,tuple):
                kk = tuple(sorted(k)) if not k==('mycophenolic acid','fenbendazole') else k
            else:
                kk = k
            for num,cwell_d in rep_well_d.items():
                if kk in cwell_d.keys():
                    for wl,vol in cwell_d[kk]:
                        if vol==40:
                            drg_data_d[kk].append(cl_data.loc[wl,(aorb,str(num))])
                            drg_inds_d[kk].append((wl,(aorb,str(num))))
                        else:
                            if isinstance(kk,tuple):
                                kkp = tuple([elt+'_ld' for elt in kk])
                            else:
                                kkp = kk+'_ld'
                            drg_data_d[kkp].append(cl_data.loc[wl,(aorb,str(num))])
                            drg_inds_d[kkp].append((wl,(aorb,str(num))))
    drg_data_d = dict(drg_data_d)
    drg_inds_d = dict(drg_inds_d)
    cl_drg_data_d[CL] = drg_data_d
    cl_drg_inds_d[CL] = drg_inds_d


In [ ]:

antisel_pairs = [elt for elt in antisel_well_d.keys() if isinstance(elt,tuple)]
nonsel_pairs = [kk for kk in nonsel_inds_d.keys() if isinstance(kk,tuple)]
nc_alpha_pairs = [elt  for elt in noncanc_alpha_d.keys() if isinstance(elt,tuple)]
nc_beta_pairs = [elt  for elt in noncanc_beta_d.keys() if isinstance(elt,tuple)]
noncanc_pairs = nc_alpha_pairs+nc_beta_pairs
ld_pairs = [elt for elt in ld_inds_d.keys() if isinstance(elt,tuple)]
colkw = 'loewe_12'
## the following drug pairs are shared between the top and non-cancer pairs 
shared_l = [('calcipotriol', 'lomefloxacin'), ('chir-99021', 'olopatadine'), ('chir-99021', 'sumatriptan'),
            ('diacerein', 'sgc-cbp30'), ('digitoxin', 'digoxin'), ('digoxin', 'pivmecillinam'),
            ('digoxin', 'sgc-cbp30'), ('miconazole', 'sulfamethoxypyridazine'), ('pitavastatin', 'pmsf'),]

top_sel = list(set(index_dict[('all','all','top')])-set(shared_l))
noncanc_l = [ kk for kk in noncanc_alpha_d.keys() if isinstance(kk,tuple)]+shared_l
lbl_kw_d = {'top':top_sel,'anti':antisel_pairs,
            'random':nonsel_pairs,'noncanc':noncanc_l}

comb_pred = pd.read_excel('%s/TableS10_combined_predictions.xlsx' % INPUT_DIR,engine='openpyxl',index_col=[0,1],header=[0,1])


### Fig. S4

#### Script to generate the mechanism of action-to-drug matrix

In [ ]:
## invert the drug --> moa mapping to moa --> drug_list
moa2drug = defaultdict(list)
single =test_grps[(test_grps.drug2_name.isna())&(test_grps.drug1_uM==10)]
for drg,moa_l in drug2moa.items():
    if isinstance(moa_l,str):
        moa2drug[moa_l].append(drg)
    else:
        for moa in moa_l:
            moa2drug[moa].append(drg)
moa2drug = dict(moa2drug)

## get counts of moa
moa_counts = {}
for k,v in moa2drug.items():
    moa_counts[k]=len(v)
moa_counts_ser = pd.Series(moa_counts)

## low dosage drugs
moa2drug_ld = defaultdict(list)
single_ld =test_grps[(test_grps.drug2_name.isna())&(test_grps.drug1_uM<10)]
for rn,row in single_ld.iterrows():
    drg = row.loc['drug1_name']
    moa_l = drug2moa[drg]
    if isinstance(moa_l,str):
        moa2drug_ld[moa_l].append(drg)
    else:
        for moa in moa_l:
            moa2drug_ld[moa].append(drg)
moa2drug_ld = dict(moa2drug)
moa_counts_ld = {}
for k,v in moa2drug_ld.items():
    moa_counts_ld[k]=len(v)
moa_counts_ld_ser = pd.Series(moa_counts_ld)

## construct matrix of (moa \times drug group)
moa2dgrp = defaultdict(list)
for moa in moa_counts_ser[moa_counts_ser>1].index:
    drg_l = moa2drug[moa]
    for drg in drg_l:
        grp =single[single.drug1_name==drg]['group'].iloc[0]
        moa2dgrp[(moa,grp)].append(drg)
    if moa in moa_counts_ld_ser.index:
        drg_ld_l = moa2drug_ld[moa]
        for drg_ld in drg_ld_l:
            moa2dgrp[(moa,'Low-dosage')].append(drg_ld)
for moa in moa_counts_ser[moa_counts_ser==1].index:
    drg = moa2drug[moa][0]
    moa2dgrp[(moa,'Other')].append(drg)
            
moa2dgrp = dict(moa2dgrp)

moadgrp_counts = {}
for kw,ll in moa2dgrp.items():
    moadgrp_counts[kw]=len(set(ll))
moa_dgrp_df = pd.Series(moadgrp_counts).unstack().fillna(0)
aaa = moa_dgrp_df.T[moa_dgrp_df.columns!='Other'].T
moa_dgrp_df2 = aaa[aaa.sum(axis=1)>0]

`moa_dgrp_df`: is the matrix that counts the number of drugs for each group

`moa_dgrp_df2`: is the matrix that counts the number of drugs for each group, excluding the "Other" category.

#### Fig. S4a

In [ ]:
srt_cols = moa_dgrp_df2.sum().sort_values(ascending=False).index
clr_l = ['C3','C4','#EDB120','#4DBEEE','#E377c2']
grplbls = ['Top selected (purple)','Non-cancer','Anti-selected','Unselected','Low-dosage']
grpclr_d = dict(zip(grplbls,clr_l))
moafig,moaax = plt.subplots(1,1,dpi=180,figsize=(4.5,3))
baseline = np.zeros(moa_dgrp_df2.shape[0])
for nm,col in moa_dgrp_df2.loc[:,srt_cols].sort_values('Low-dosage',ascending=False).items():
    if nm=='Other':
        continue
    lbl = nm.split(' (')[0] if 'purple' in nm else nm    
    #scol = col[(moa_dgrp_df.Other-moa_dgrp_df.sum(axis=1))<0]
    moaax.bar(np.arange(col.shape[0]),col,bottom=baseline,label=lbl,color=grpclr_d[nm])
    baseline+=col.values
moaax.set_xticks(np.arange(moa_dgrp_df2.shape[0]))
moaax.set_xticklabels(moa_dgrp_df2.sort_values('Low-dosage',ascending=False).index,size=6,rotation=45,horizontalalignment='right')
moaax.set_yticks(np.arange(0,21,5))
moaax.set_yticklabels(np.arange(0,21,5),size=6)
moaax.set_xlim(-0.75,moa_dgrp_df2.shape[0]-0.25)
moaax.set_xlabel('Mechanism of action',size=8)
moaax.set_ylabel('Count',size=8)
#moafig.savefig('%s/figS4a_moa_counts.svg'% OUTPUT_DIR)

#### Fig. S4a (inset)

In [ ]:
moafig2 = plt.figure(figsize=(3,1.5),dpi=180)
moaax2 = moafig2.add_axes([.15,.2,.8,.75])
colsums = moa_dgrp_df2.sum()

xlbls = []
for ii,(nm,col) in enumerate(colsums.sort_values(ascending=False).items()):
    lbl = nm.split(' (')[0] if 'purple' in nm else nm
    xlbls.append(lbl)
    moaax2.bar([ii],[col],color=grpclr_d[nm])
    
moaax2.set_xticks(np.arange(len(xlbls)))
moaax2.set_xticklabels(xlbls,size=6,rotation=45,horizontalalignment='right')

moaax2.set_xlabel('Drug group',size=8)
moaax2.set_ylabel('Count',size=8)
#moafig2.savefig('%s/figS4a_inset_drug_group_counts_inset.svg'% OUTPUT_DIR)

In [ ]:
## plot the viability data
moa2dgrp_all = defaultdict(list)
for moa in moa_counts_ser.index:
    drg_l = moa2drug[moa]
    for drg in drg_l:
        grp =single[single.drug1_name==drg]['group'].iloc[0]
        moa2dgrp_all[(moa,grp)].append(drg)
    if moa in moa_counts_ld_ser.index:
        drg_ld_l = moa2drug_ld[moa]
        for drg_ld in drg_ld_l:
            moa2dgrp_all[(moa,'Low-dosage')].append(drg_ld)
            
moa2dgrp_all = dict(moa2dgrp_all)

ii = 0
sel_df_comb = sel_df_comb.T[sel_df_comb.columns.get_level_values(0)!='12'].T ## viability data (obtained earlier)
dgrp_viability_d = defaultdict(list) ## container for the viability data
grplbls2 = ['Low-dosage','Top selected (purple)','Non-cancer','Anti-selected','Unselected','Other']
grpclr_d['Other']='#808080'
dgrp_shift_d = dict(zip(grplbls2,0.1*np.arange(len(grplbls2))))
moa2xval = dict([(yy,xx) for xx,yy in enumerate(moa_dgrp_df2.sort_values('Low-dosage',ascending=False).index)])
cl_title_d = {'43':'GBM43','LN':'LN-229','CH':'CH-157MN','Io':'IOMM-LEE'}
moa2xval['Other']=len(moa2xval.keys())
for (moa,dgrp),drg_l in moa2dgrp_all.items():
    layout = single if dgrp != 'Low-dosage' else single_ld
    if moa in moa2xval.keys():
        for drg in drg_l:
            if not drg in layout.drug1_name.unique():
                continue
            pl,wl = layout[layout.drug1_name==drg].index[0]
            pp = 'A' if pl=='alpha' else 'B'
            avg_via_cl = sel_df_comb.xs(pp,axis=1,level=1).loc[wl].groupby(level=0).mean()
            std_via_cl = sel_df_comb.xs(pp,axis=1,level=1).loc[wl].groupby(level=0).std()
            for cl,via in avg_via_cl.items():
                dgrp_viability_d[(cl,dgrp,'xvals')].append(moa2xval[moa]+dgrp_shift_d[dgrp]+np.random.rand()*1e-2)
                dgrp_viability_d[(cl,dgrp,'yvals')].append(via)
                dgrp_viability_d[(cl,dgrp,'ystd')].append(std_via_cl.loc[cl])
    else:
        for drg in drg_l:
            if not drg in layout.drug1_name.unique():
                continue
            pl,wl = layout[layout.drug1_name==drg].index[0]
            pp = 'A' if pl=='alpha' else 'B'
            avg_via_cl = sel_df_comb.xs(pp,axis=1,level=1).loc[wl].groupby(level=0).mean()
            std_via_cl = sel_df_comb.xs(pp,axis=1,level=1).loc[wl].groupby(level=0).std()
            for cl,via in avg_via_cl.items():
                dgrp_viability_d[(cl,'Other','xvals')].append(moa2xval['Other']+dgrp_shift_d[dgrp]+np.random.rand()*1e-2)
                dgrp_viability_d[(cl,'Other','yvals')].append(via)
                dgrp_viability_d[(cl,'Other','ystd')].append(std_via_cl.loc[cl])
dgrp_viability_d = dict(dgrp_viability_d)

#### Fig. S4b

In [ ]:
fig,ax_l = plt.subplots(4,1,dpi=300,sharex=True,figsize=(7.5,5))
for ii,(cl,title) in enumerate(sorted(cl_title_d.items(),key=lambda I: I[1])):
    ax = ax_l[ii]
    for dgrp in grplbls2:
        lbl = dgrp if not '(' in dgrp else dgrp.split(' (')[0]
        xvals = dgrp_viability_d[(cl,dgrp,'xvals')]
        yvals = dgrp_viability_d[(cl,dgrp,'yvals')]
        yerr = dgrp_viability_d[(cl,dgrp,'ystd')]
        #ax.errorbar(xvals,yvals,yerr=yerr,ls='',marker='.',ecolor=grpclr_d[dgrp],mfc=grpclr_d[dgrp],mec='none',label=dgrp)
        ax.scatter(xvals,yvals,marker='.',color=grpclr_d[dgrp],alpha=0.5,label=lbl)
    ax.set_title(title,size=10)
    ax.set_ylabel('Viability',size=8)
    ax.set_yscale('log')
    if ii==0:
        leg = ax.legend(frameon=False,prop={'size':6},ncol=3)
    
ax.set_xticks(np.arange(len(moa2xval.keys())))
ax.set_xticklabels(moa2xval.keys(),size=6,rotation=45,horizontalalignment='right')
ax.set_xlabel('Mechanism of action',size=8)
#fig.savefig('%s/figS4b_data.svg'% OUTPUT_DIR)

#### Mechanism of Action statistics

In [ ]:
## get the drug to moa matrix, needed for constructing the statistical model. "A" in Eq. (S26)
drg_moa_matrix = defaultdict(int)
for drg,moa_l in drug2moa.items():
    if isinstance(moa_l,str):
        MOA='Other' if moa_counts_ser.loc[moa_l]==1 else moa_l
        drg_moa_matrix[(drg,MOA)]+=1
    else:
        for moa in moa_l:
            MOA='Other' if moa_counts_ser.loc[moa]==1 else moa
            drg_moa_matrix[(drg,MOA)]+=1
                
    
drg_moa_matrix = dict(drg_moa_matrix)
drg_moa_matrix2 = pd.Series(drg_moa_matrix).unstack().fillna(0)
drg_moa_matrix2

drg_avgvia_d = {} ## average viability measurements
for drg in single.drug1_name.unique():
    pl,wl = single[single.drug1_name==drg].index[0]
    pp = 'A' if pl=='alpha' else 'B'
    avg_via_cl = sel_df_comb.xs(pp,axis=1,level=1).loc[wl].groupby(level=0).mean()
    drg_avgvia_d[drg]=np.log(avg_via_cl)
drg_avgvia_df = pd.DataFrame(drg_avgvia_d)
Q = drg_moa_matrix2.T@drg_moa_matrix2
C = (drg_avgvia_df@drg_moa_matrix2).T
u = np.linalg.solve(Q,C)
est_vals = drg_moa_matrix2@u ## estimated viability based on mechanism of action
est_vals.columns = drg_avgvia_df.index
act_rsqr = 1-(drg_avgvia_df.T-est_vals).var()/drg_avgvia_df.T.var() ## actual R^2

In [ ]:
## randomly reshuffle dat
cpy = drg_avgvia_df.copy()
rnd_rsqr_l = []
for ii in range(1000):
    np.random.shuffle(cpy.T.values)
    C_rnd = (cpy@drg_moa_matrix2).T
    u_rnd = np.linalg.solve(Q,C_rnd)
    est_vals_rnd = drg_moa_matrix2@u_rnd
    est_vals_rnd.columns = drg_avgvia_df.index
    rnd_rsqr = 1-(cpy.T-est_vals_rnd).var()/cpy.T.var()
    rnd_rsqr_l.append(rnd_rsqr)
rnd_rsqr_df = pd.concat(rnd_rsqr_l,axis=1) ## random R^2 distribution.

In [ ]:
## the next line prints out the p-values
pd.Series(norm.sf((act_rsqr-rnd_rsqr_df.mean(axis=1))/rnd_rsqr_df.std(axis=1))*2,index=[cl_title_d[cc] for cc in act_rsqr.index]).sort_index()

#### Fig. S4c

In [ ]:
srt_clks = [clk for clk,clt in sorted(cl_title_d.items(),key=lambda it: it[1])]
rnd_rsqr_df = rnd_rsqr_df.loc[srt_clks]
fig,ax = plt.subplots(1,1,dpi=180,figsize=(1.5,4))
ax.boxplot([rnd_rsqr_df.iloc[ii,:] for ii in range(rnd_rsqr_df.shape[0])],
           positions=np.arange(rnd_rsqr_df.shape[0]),
           widths=np.ones(rnd_rsqr_df.shape[0])*0.8,
          )
ax.plot(np.arange(act_rsqr.shape[0]),act_rsqr.loc[srt_clks],ls='',marker='*')
ax.set_xticklabels(sorted(cl_title_d.values()),rotation=30,size=6,horizontalalignment='right')
plt.setp(ax.get_yticklabels(),size=6)
ax.set_xlabel('Cell line')
ax.set_ylabel(r'$R^2_{\rm MOA}$')
#fig.savefig('%s/figS4c_stat_test_boxplot.svg'% OUTPUT_DIR)

### Figs. 3c & 5a (venn diagrams)

#### Fig. 3c

In [ ]:
set_d = {}
for (meth,vari),col in comb_pred.items():
#     vari_nm = vari_nm_d[vari]
    nz_col = col[~col.isna()]
    inds = [tuple(sorted(elt)) for elt in nz_col.index.tolist()]
    set_d[(meth,vari)]=set(inds)

fig,ax_l = plt.subplots(1,2,dpi=180)
method_lbl_d = {'0.00':'Behavior','0.79':'Blended','1.57':'Growth'}
for yy,tp in enumerate(['nonadditive','additive']):    
    venn3([set(vv) for kk,vv in set_d.items() if kk[0]==tp],
          [method_lbl_d[kk[1]] for kk in set_d.keys() if kk[0]==tp],
          ['C%d' % xx for xx in range(3)],alpha=0.5,ax=ax_l[yy])
    ax_l[yy].set_title(tp)
fig.savefig('%s/fig3c_additive_nonadditive_venn_diagrams_exp.svg' % OUTPUT_DIR)

#### Fig. 5a

In [ ]:
fig,ax_l = plt.subplots(1,2,dpi=180)
method_lbl_d = {'0.00':'Behavior','0.79':'Blended','1.57':'Growth'}
for yy,tp in enumerate(['additive','nonadditive']):    
    venn3([set(vv) for kk,vv in set_d.items() if kk[0]==tp],
          [method_lbl_d[kk[1]] for kk in set_d.keys() if kk[0]==tp],
          ['C%d' % xx for xx in range(3)],alpha=0.5,ax=ax_l[yy])
fig.savefig('%s/fig5a_additive_nonadditive_venn_diagrams.svg' % OUTPUT_DIR) ## (this figures is not in the paper)
fig2,ax_l2 = plt.subplots(1,3,dpi=180)
for zz,mthd in enumerate(['0.00','0.79','1.57']):
    venn2([set(vv) for kk,vv in set_d.items() if kk[1]==mthd],
         [kk[0] for kk in set_d.keys() if kk[1]==mthd],
         ['C%d' % zz for dum in range(2)],alpha=0.5,ax=ax_l2[zz])
fig2.savefig('%s/fig5a_method_venn_diagrams.svg' % OUTPUT_DIR)

### Fig. 5cde (AUC example & method boxplots)

#### Fig. 5c

In [ ]:
fnbase_l = ['lodo_mu_sig_%s_dataset.pkl','ctpair_3fold_mu_sig_%s_celltypepair.pkl']
DATA = pd.DataFrame(cl_drg_data_d['CH-157MN']).T
nonadd_IS = comb_pred.loc[:,('nonadditive','0.79')].sort_values(ascending=False).index.intersection(DATA.index)
add_IS = comb_pred.loc[:,('additive','0.79')].sort_values(ascending=False).index.intersection(DATA.index)
v1 = DATA.loc[nonadd_IS].mean(axis=1)
v1r = comb_pred.loc[:,('nonadditive','0.79')].rank(ascending=False,pct=True).loc[nonadd_IS]
v2 = DATA.loc[add_IS].mean(axis=1)
v2r = comb_pred.loc[:,('additive','0.79')].rank(ascending=False,pct=True).loc[add_IS]
sel_v1 = v1[~v1r.isna()]
sel_v2 = v2[~v2r.isna()]
common = sel_v1.index.intersection(sel_v2.index)
def alphasrt_multiindex(MI):
    alphasrt_inds = []
    for (a,b) in MI:
        if a<b:
            alphasrt_inds.append((a,b))
        else:
            alphasrt_inds.append((b,a))
    return pd.MultiIndex.from_tuples(alphasrt_inds)
NRREP = 100

auc_info_d = {}
for CL,drg_data_d in cl_drg_data_d.items():
    DATA = pd.DataFrame(drg_data_d).T
    sel_TF = [isinstance(ind,tuple) and not ind[0].endswith('_ld') for ind in DATA.index]
    DATA.index = pd.MultiIndex.from_tuples([k if isinstance(k,tuple) else (k,'dmso') for k in DATA.index])
    #VTHR=DATA.mean(axis=1).describe().loc['50%']
    VTHR= DATA[sel_TF].mean(axis=1).median()
    for fnbase in fnbase_l[:1]:
        lbl = 'lodo' if fnbase.startswith('lodo') else 'ctpair'
        for objval in ['0.00','0.79','1.57']:
            if CL=='LN-229' and objval=='0.79':
                fig,ax = plt.subplots(2,1,dpi=180,figsize=(1.5,4))
            fpr_l = []
            tpr_l = []
            thrl_l = []
            IS_l = []
            var_pred_l = []
            for kk in range(2):
                bsubstr,predstr = ('nonadd','nonadditive') if kk!=0 else ('linear','additive')
                fntot = INPUT_DIR + '/' + fnbase
                xdiffs = pd.read_pickle(fntot % bsubstr)
                cmb_diffs = xdiffs.xs(objval,level=2).copy()
                pred = comb_pred.loc[:,(predstr,objval)]
                #pred = pred[~pred.isna()]
                cmb_diffs.index = alphasrt_multiindex(cmb_diffs.index)
                IS = pred[~pred.isna()].index.intersection(DATA.index)
                IS_l.append(IS)
                var_pred_l.append((cmb_diffs,pred,predstr))
            common = IS_l[0].union(IS_l[1])
            common_stats = DATA.loc[common].T.describe().T.loc[:,'mean':'std']
            rnd_vals_l = []
            auc_dd = defaultdict(list)
            for kk,(cmb_diffs,pred,predstr) in enumerate(var_pred_l):
                fact= 6 if fnbase.startswith('lodo') and kk==0 else 1
                duma = pred.loc[common].sort_values(ascending=False) #.fillna(0)
                wtrank = (duma/duma.sum()).cumsum()
                wtrank_l = []
                for elt in common.difference(cmb_diffs.index):
                    cmb_diffs.loc[elt,'mu'] = np.nan
                    cmb_diffs.loc[elt,'sig'] = np.nan
                for nn in range(NRREP):
                    rnd_val_d = {}
                    dum = pred.loc[common]+fact*cmb_diffs.loc[common,'sig']*np.random.randn(common.shape[0])
                    rnd_wtrank = (dum/dum.sum()).sort_values(ascending=False).cumsum().loc[common]
                    if kk==0:
                        rnd_vals = common_stats.loc[common,'mean']+np.random.randn(common_stats.shape[0])*common_stats.loc[common,'std']
                        rnd_vals_l.append(rnd_vals)
                    else:
                        rnd_vals = rnd_vals_l[nn]
                    SEL = ~rnd_wtrank.loc[common].isna()
                    rfpr,rtpr,rthrl = roc_curve(common_stats[SEL].loc[:,'mean']<VTHR,1-rnd_wtrank[SEL])
                    auc_dd[predstr].append(auc(rfpr,rtpr))
                    wtrank_l.append(rnd_wtrank)
                    
                    ## need to calculate the AUC distribution here
                wtrank_sig = pd.concat(wtrank_l,axis=1).std(axis=1)/10.
                fpr,tpr,thrl = roc_curve(common_stats[~wtrank.loc[common].isna()].loc[:,'mean']<VTHR,1-wtrank.loc[common][~wtrank.isna()])
                fpr_l.append(fpr)
                tpr_l.append(tpr)
                thrl_l.append(thrl)
                act_auc = auc(fpr,tpr)
                auc_dd[predstr].append(act_auc)
                if CL=='LN-229' and objval=='0.79':
                    ax[0].errorbar(wtrank.loc[common],common_stats.loc[:,'mean'],
                                 xerr=wtrank_sig.loc[common],yerr=common_stats.loc[:,'std'],fmt='C%d.' % (kk+3),ms=4,alpha=0.5)
                    ax[1].plot(fpr,tpr,label='%s : %.02f' % (predstr,act_auc),color='C%d' % (kk+3))
            auc_d = dict(auc_dd)
            for kk,vv in auc_d.items():
                auc_info_d[(CL,objval,kk)]=vv
            if CL=='LN-229' and objval=='0.79':
                ax[0].set_yscale('log')
                ax[0].set_xticks([0,0.5,1])
                ax[0].set_ylabel('Viability',size=8)
                ax[0].set_xlabel('Weighted rank',size=8)
                ax[0].axhline(VTHR,ls='--',color='C7')
                ind1 = 4
                ax[0].axvline((1-thrl_l[1][ind1]),ls='-',color='C7')
                plt.setp(ax[0].get_xticklabels(),size=6)
                plt.setp(ax[0].get_yticklabels(),size=6)
                ax[1].set_ylabel('TPR',size=8)
                ax[1].set_xticks([0,0.5,1])
                ax[1].set_xlabel('FPR',size=8)
                ax[1].plot(fpr_l[1][ind1:ind1+1],tpr_l[1][ind1:ind1+1],color='C4' ,marker=(5,1,0))
                ind0 = np.argmin(np.abs((1-np.asarray(thrl_l[0]))-(1-thrl_l[1][ind1])))
                ax[1].plot(fpr_l[0][ind0:ind0+1],tpr_l[0][ind0:ind0+1],color='C3',marker=(5,1,0))                 
                ax[1].legend(title='AUC',frameon=False,prop={'size':6})
                ax[1].plot(np.linspace(0,1),np.linspace(0,1),ls='--',color='C7')
                plt.setp(ax[1].get_xticklabels(),size=6)
                plt.setp(ax[1].get_yticklabels(),size=6)
                #fig.tight_layout()
                #fig.savefig('%s/fig5c_example_LN-229_0.79_trend.svg' % OUTPUT_DIR)
kw_df = pd.DataFrame(auc_info_d.keys())

#### Fig. 5d

In [ ]:
fig2,ax2a = plt.subplots(1,4,dpi=180,figsize=(3.5,1.5),sharey=True,sharex=True)
objval_clr_d = {'0.00':'C0','0.79':'C1','1.57':'C2'}
objval_str_d = {'0.00':'Behavior','0.79':'Blended','1.57':'Growth'}
objval_num_d = {'0.00':0,'0.79':1,'1.57':2}
kk_num_d = {'additive':0,'nonadditive':1}
auc_tstat_d = {}
for ii,CL in enumerate(kw_df.iloc[:,0].unique()):    
    ax2 = ax2a[ii]
    ax2.set_title(CL,size=8)
    if ii==0:
        ax2.set_ylabel('AUC',size=8)
        ax2.set_ylim(0,1)
        plt.setp(ax2.get_yticklabels(),size=6)
    clr_dat_d = defaultdict(list)
    for (CLz,objval,kk) in kw_df[kw_df.iloc[:,0]==CL].values:
        clr = objval_clr_d[objval]
        pos=2*objval_num_d[objval]+kk_num_d[kk]
        clr_dat_d[clr].append((auc_info_d[(CL,objval,kk)],pos,kk))
    for (CLx,objval1,kk1),(CLy,objval2,kk2) in combs(kw_df[kw_df.iloc[:,0]==CL].values,2):
        auc_l1 = auc_info_d[(CL,objval1,kk1)]
        auc_l2 = auc_info_d[(CL,objval2,kk2)]
        tstat,pval = ranksums(auc_l1,auc_l2)
        auc_tstat_d[(CL,objval1,kk1,objval2,kk2,'rstat')]=tstat
        auc_tstat_d[(CL,objval1,kk1,objval2,kk2,'pval')]=pval
    clr_dat_d = dict(clr_dat_d)
    clr_l = []
    box_l = []
    for clr,dat in clr_dat_d.items():
        data_l,pos_l,lbl_l = zip(*dat)
        box = ax2.boxplot(data_l,positions=pos_l,labels=lbl_l,widths=0.6,patch_artist=True,
                         notch=False,sym='%s.' % clr,boxprops={'color':clr,'alpha':0.5},
                         whiskerprops={'color':'k'}, capprops={'color':'k'},
                         medianprops={'color':'C3'},flierprops={'alpha':0.5,'markeredgecolor':None})
        box_l.append(box)
        clr_l.append(clr)
    for clr,bplot in zip(clr_l,box_l):
        sclr = [box.set_color(clr) for box in bplot['boxes']]
        sal = [box.set_alpha(0.5) for box in bplot['boxes']]
    plt.setp(ax2.get_xticklabels(),rotation=90,size=6)
fig2.savefig('%s/fig5d_ranked_viability_auc.svg' % OUTPUT_DIR)
auc_tstat_ser=pd.Series(auc_tstat_d)
auc_tstat_df = auc_tstat_ser.unstack(0)
auc_tstat_df.to_excel('%s/ranked_viability_auc_tstats.xlsx' % OUTPUT_DIR)

### Code to calculate the synergy/${\rm IC}_{50}$ statistics

In [ ]:
## v_{LA} threshold for the change in viability
BP_THR = 1e-2
colkw = 'loewe_12' ## type of synergy estimate
## container for the cell line & synergy statistics
cl_synstat_d = {}
## function to describe drug-response curves
def ic50_func(x,a):
    return 1/((x/a)+1)

for CL,data_d in cl_drg_data_d.items():
    DATA = pd.DataFrame(data_d).T
    DATA.index = pd.MultiIndex.from_tuples([k if isinstance(k,tuple) else (k,'dmso') for k in DATA.index])
    ## calculate the IC50s
    ## multiple dosages
    xic50_fits = {}
    for drg in reduced_dosage_d.keys():
        ## case 1 drug and drug_ld both exist in the dataset
        yvals1 = np.clip(DATA.loc[(drg,'dmso')],1e-6,1-1e-6).values
        yvals2 = np.clip(DATA.loc[(drg+'_ld','dmso')],1e-6,1-1e-6).values
        xvals1 = np.ones(yvals1.shape[0])*10
        xvals2 = np.ones(yvals2.shape[0])*10 * reduced_dosage_d[drg]/40.
        vrow_fit = np.r_[yvals1,yvals2]
        crow_fit = np.r_[xvals1,xvals2]
        if np.amin(vrow_fit)< 0.5 and 0.5 <np.amax(vrow_fit):
            guess = np.clip(np.median(crow_fit),0,200)
        elif np.amax(vrow_fit)< 0.5:
            guess = np.clip(np.amax(crow_fit),0,200)
        elif np.amin(vrow_fit)> 0.5:
            guess = np.clip(np.amin(crow_fit),0,200)
        aa,bb = curve_fit(ic50_func,crow_fit,vrow_fit,guess,bounds=(1e-3,1e3))
        #aa,bb = curve_fit(ic50_logfunc,crow_fit,np.log(vrow_fit),guess)
        xic50_fits[(drg,'ic50')]=aa[0]
        xic50_fits[(drg,'ic50_std')]=np.sqrt(bb[0][0])
    xic50_fits = pd.Series(xic50_fits)
    ## single dosages
    myic50s = {}
    for drg,row in DATA.xs('dmso',level=1).iterrows():
        if row.mean(axis=0)>.95:
            myic50s[(drg,'mu')]=2e2
            myic50s[(drg,'sig')]=0
        elif drg in xic50_fits.index.get_level_values(0):
            myic50s[(drg,'mu')] = xic50_fits.xs('ic50',level=1).loc[drg]
            myic50s[(drg,'sig')] = xic50_fits.xs('ic50_std',level=1).loc[drg]
        else:
            myic50s[(drg,'mu')]=10/(1/row.mean()-1)    
            myic50s[(drg,'sig')]=np.std([10/(1/(row.mean()+row.std()*np.random.randn())-1) for nn in range(100)])
    myic50_df = pd.Series(myic50s).unstack()
    SEL_SINGLES = DATA.xs('dmso',level=1)
    SEL_DOUBLES = DATA[DATA.index.get_level_values(1)!='dmso']
    drgp_syn_d = {}
    ## calculate the drug synergy
    for (drg1,drg2),row in SEL_DOUBLES.iterrows():
        drgp_syn_d[(drg1,drg2,'v_12')] = row.mean(axis=0)
        drgp_syn_d[(drg1,drg2,'v_12_std')] = row.std(axis=0)
        v_1 = SEL_SINGLES.loc[drg1].mean(axis=0)
        v_1_std = SEL_SINGLES.loc[drg1].std(axis=0)
        v_2 = SEL_SINGLES.loc[drg2].mean(axis=0)
        v_2_std = SEL_SINGLES.loc[drg2].std(axis=0)
        drgp_syn_d[(drg1,drg2,'v_1')] = v_1
        drgp_syn_d[(drg1,drg2,'v_1_std')] = v_1_std
        drgp_syn_d[(drg1,drg2,'v_2')] = v_2
        drgp_syn_d[(drg1,drg2,'v_2_std')] = v_2_std
        drgp_syn_d[(drg1,drg2,'bliss')] = v_1*v_2
        drgp_syn_d[(drg1,drg2,'bliss_std')] = np.sqrt(v_1_std**2+v_2_std**2)
        if drg1.endswith('_ld'):
            drg1p = drg1.split('_')[0]
            drg2p = drg2.split('_')[0]
            drgp_syn_d[(drg1,drg2,'loewe_12')] = 1/((10*reduced_dosage_d[drg1p]/40)/myic50_df.loc[drg1p,'mu']+(10*reduced_dosage_d[drg2p]/40)/myic50_df.loc[drg2p,'mu']+1)
        else:
            drgp_syn_d[(drg1,drg2,'loewe_12')] = 1/(10/myic50_df.loc[drg1,'mu']+10/myic50_df.loc[drg2,'mu']+1)
    synstat_df = pd.Series(drgp_syn_d).unstack()
    cl_synstat_d[CL]=synstat_df

In [ ]:
## need to do fig. 5e (low-dosage analysis)
totaldosage=pd.Series(dict([((elt1+'_ld',elt2+'_ld'),reduced_dosage_d[elt1]+reduced_dosage_d[elt2]) for elt1,elt2 in ld_pairs]))
ldstat_d ={}
xTHR=1e-2 ## threshold to be included in "top" ranked group
tb_spc = 0.5
fig,ax_l = plt.subplots(1,4,dpi=180,figsize=(6.11,1.82),sharey=True)
colkw ='loewe_12'
clr_d = defaultdict(dict)
objval_clr_d = {'0.00':'C0','0.79':'C1','1.57':'C2'}
sym_d = {'Top':'^','Bottom':'v'}
num_d = {'0.00':0,'0.79':2,'1.57':4,'additive':0,'nonadditive':1}
for ar,CL in enumerate(['CH-157MN','GBM43','IOMM-LEE','LN-229']):
    print(CL)
    syn_d = defaultdict(dict)
    lbl_d = defaultdict(dict)
    pos_d = defaultdict(dict)
    for col in comb_pred.columns:
        synstat_df = cl_synstat_d[CL]
        high_thr = (synstat_df.loc[:,colkw]/synstat_df.v_12>1)[synstat_df.loc[:,colkw]>BP_THR].mean()
        low_thr = (synstat_df.loc[:,colkw]/synstat_df.v_12>1)[synstat_df.loc[:,colkw]<BP_THR].mean()
        sum_pred_ranks = comb_pred.loc[:,col].fillna(0).rank(pct=True,ascending=False)
        for kw in ['Top','Bottom']:
            if kw == 'Top':
                tot_sp = [elt for elt in ld_pairs if sum_pred_ranks.loc[elt]<xTHR]
            else:
                tot_sp = [elt for elt in ld_pairs if sum_pred_ranks.loc[elt]>=xTHR]

            sel_ld_keys = [tuple([val+'_ld' for val in elt]) for elt in tot_sp]
            exp_syn = ((synstat_df.loc[sel_ld_keys,'loewe_12']<BP_THR)*low_thr+(synstat_df.loc[sel_ld_keys,'loewe_12']>BP_THR)*high_thr)
            obs_syn = (synstat_df.loc[sel_ld_keys,'v_12']<synstat_df.loc[sel_ld_keys,'loewe_12']).mean()
            rnd_avg_l = []
            ## bootstrap the synergy values
            for dum in range(100):
                rnd_obs = synstat_df.loc[sel_ld_keys,'v_12']+np.random.randn(len(sel_ld_keys))*synstat_df.loc[sel_ld_keys,'v_12']
                rnd_avg = (rnd_obs<synstat_df.loc[sel_ld_keys,'loewe_12']).mean()
                rnd_avg_l.append(rnd_avg)
            rat_sig = np.std(rnd_avg_l)/exp_syn.mean() / np.sqrt(exp_syn.shape[0])
            rat_syn = obs_syn/exp_syn.mean()
            syn_d[col[1]][(kw,col[0])] = (rat_syn,rat_sig,exp_syn.shape[0])
            pos_ind = num_d[col[1]]+num_d[col[0]]
            pos_d[col[1]][(kw,col[0])] = (pos_ind)
            if kw=='Top':
                lbl_d[col[1]][(kw,col[0])] = ('%s %s' % (kw,col[0]))
    syn_d = dict(syn_d)
    pos_d = dict(pos_d)
    xticks=[]
    xticklabels=[]
    ## need to adjust to the new format of the syn_d dictionary 
    ## so that there is a legend with different symbols
    for objval,method_dat_d in syn_d.items():
        clr = objval_clr_d[objval]
        syn_dat_d = syn_d[objval]
        for (torb,aorna),(rats,raterrs,nums) in syn_dat_d.items():
            coll = ax_l[ar].errorbar(pos_d[objval][(torb,aorna)],rats,yerr=raterrs,fmt='%s%s' % (clr,sym_d[torb]),label=torb)
            if torb=='Top':
                xticks.append(pos_d[objval][(torb,aorna)] + tb_spc/2.)
                xticklabels.append(lbl_d[objval][(torb,aorna)])
            
        for ((meth1,tob1),syn1),((meth2,tob2),syn2) in combs(syn_dat_d.items(),2):
            mu1,sig1,obs1 = syn1
            mu2,sig2,obs2 = syn2
            rss2,rsp2 =ttest_ind_from_stats(mu1,sig1,obs1,mu2,sig2,obs2)
            ldstat_d[(CL,objval,meth1,tob1,meth2,tob2,'syn','rs_stat')]=rss2
            ldstat_d[(CL,objval,meth1,tob1,meth2,tob2,'syn','rs_pval')]=rsp2
    
    #ax_l[ar].set_title(CL,size=8)
    plt.setp(ax_l[ar].get_yticklabels(),size=6)
    ax_l[ar].axhline(1,ls='--',color='C7')

    ax_l[ar].set_xticks(xticks)
    ax_l[ar].set_xticklabels(xticklabels,size=6,rotation=90)
    plt.setp(ax_l[ar].get_xticklabels(),size=6,rotation=90)
ax_l[0].set_ylim(0.95,1.75)
plt.setp(ax_l[0].get_yticklabels(),size=6)
#ax_l[0].axhline(1,ls='--')
ax_l[0].set_ylabel('Synergy ratio',size=8)
ax_l[-1].legend(frameon=False,prop={'size':6})
fig.savefig('%s/fig5e_low_dosage_comparison.svg'% OUTPUT_DIR)
pd.Series(ldstat_d).unstack([-2,-1]).to_excel('%s/low_dosage_statistics.xlsx'% OUTPUT_DIR)

### Fig. S1

#### Fig. S1ab

In [ ]:
auc_syninfo_d = {}
NRREP=100
for CL,synstat_df in cl_synstat_d.items():
    VTHR=(synstat_df.loc[:,'v_12']-synstat_df.loc[:,colkw]).median()
    for fnbase in fnbase_l[:1]:
        lbl = 'lodo' if fnbase.startswith('lodo') else 'ctpair'
        for objval in ['0.00','0.79','1.57']:
            if CL=='LN-229' and objval=='0.79':
                fig,ax = plt.subplots(2,1,dpi=180,figsize=(3,4))
            fpr_l = []
            tpr_l = []
            thrl_l = []
            IS_l = []
            var_pred_l = []
            for kk in range(2):
                bsubstr,predstr = ('nonadd','nonadditive') if kk!=0 else ('linear','additive')
                fntot = INPUT_DIR + '/' + fnbase
                xdiffs = pd.read_pickle(fntot % bsubstr)
                cmb_diffs = (xdiffs.xs(objval,level=2)).copy()
                pred = comb_pred.loc[:,(predstr,objval)]
                #pred = pred[~pred.isna()]
                cmb_diffs.index = alphasrt_multiindex(cmb_diffs.index)
                IS = pred[~pred.isna()].index.intersection(synstat_df.index)
                IS_l.append(IS)
                var_pred_l.append((cmb_diffs,pred,bsubstr))
            common = IS_l[0].union(IS_l[1])
            common_stats = {}
            common_stats['mean']=(synstat_df.loc[common,'v_12']-synstat_df.loc[common,'loewe_12'])
            common_stats['std']=synstat_df.loc[common,'v_12_std']
            common_stats = pd.DataFrame(common_stats)
            rnd_vals_l = []
            auc_dd = defaultdict(list)
            for kk,(cmb_diffs,pred,bsubstr) in enumerate(var_pred_l):
                fact= 6 if fnbase.startswith('lodo') and kk==0 else 1
                duma = pred.loc[common].sort_values(ascending=False) #.fillna(0)
                wtrank = (duma/duma.sum()).cumsum()
                wtrank_l = []
                for elt in common.difference(cmb_diffs.index):
                    cmb_diffs.loc[elt,'mu'] = np.nan
                    cmb_diffs.loc[elt,'sig'] = np.nan
                for nn in range(NRREP):
                    rnd_val_d = {}
                    dum = pred.loc[common]+fact*cmb_diffs.loc[common,'sig']*np.random.randn(common.shape[0])
                    rnd_wtrank = (dum/dum.sum()).sort_values(ascending=False).cumsum().loc[common]
                    if kk==0:
                        rnd_vals = common_stats.loc[common,'mean']+np.random.randn(common_stats.shape[0])*common_stats.loc[common,'std']
                        rnd_vals_l.append(rnd_vals)
                    else:
                        rnd_vals = rnd_vals_l[nn]
                    SEL = ~rnd_wtrank.loc[common].isna()
                    rfpr,rtpr,rthrl = roc_curve(common_stats[SEL].loc[:,'mean']<VTHR,1-rnd_wtrank[SEL])
                    auc_dd[bsubstr].append(auc(rfpr,rtpr))
                    wtrank_l.append(rnd_wtrank)
                    
                    ## need to calculate the AUC distribution here
                wtrank_sig = pd.concat(wtrank_l,axis=1).std(axis=1)/10.
                fpr,tpr,thrl = roc_curve(common_stats[~wtrank.loc[common].isna()].loc[:,'mean']<VTHR,1-wtrank.loc[common][~wtrank.isna()])
                fpr_l.append(fpr)
                tpr_l.append(tpr)
                thrl_l.append(thrl)
                act_auc = auc(fpr,tpr)
                auc_dd[bsubstr].append(act_auc)
                if CL=='LN-229' and objval=='0.79':
                    ax[0].errorbar(wtrank.loc[common],common_stats.loc[:,'mean'],
                                 xerr=wtrank_sig.loc[common],yerr=common_stats.loc[:,'std'],fmt='C%d.' % (kk+3),alpha=0.5)
                    ax[1].plot(fpr,tpr,label='%s : %.02f' % (bsubstr,act_auc),color='C%d' % (kk+3))
            auc_d = dict(auc_dd)
            for kk,vv in auc_d.items():
                auc_syninfo_d[(CL,objval,kk)]=vv
            if CL=='LN-229' and objval=='0.79':
                #ax[0].set_yscale('log')
                ax[0].set_ylabel('Synergy',size=8)
                ax[0].set_xlabel('Weighted rank',size=8)
                ax[0].axhline(VTHR,ls='--',color='C7')
                ind1 = 20
                ax[0].axvline((1-thrl_l[1][ind1]),ls='-',color='C7')
                ax[0].set_ylim(-0.25,0.25)
                ax[1].set_ylabel('TPR',size=8)
                ax[1].set_xlabel('FPR',size=8)
                ax[1].plot(fpr_l[1][ind1:ind1+1],tpr_l[1][ind1:ind1+1],color='C4' ,marker=(5,1,0))
                ind0 = np.argmin(np.abs((1-np.asarray(thrl_l[0]))-(1-thrl_l[1][ind1])))
                ax[1].plot(fpr_l[0][ind0:ind0+1],tpr_l[0][ind0:ind0+1],color='C3',marker=(5,1,0))                 
                ax[1].legend(title='AUC',frameon=False,prop={'size':6})
                ax[1].plot(np.linspace(0,1),np.linspace(0,1),ls='--',color='C7')
                plt.setp(ax[0].get_xticklabels(),size=6)
                plt.setp(ax[0].get_yticklabels(),size=6)
                plt.setp(ax[1].get_xticklabels(),size=6)
                plt.setp(ax[1].get_yticklabels(),size=6)
                fig.savefig('%s/figS1a_example_LN-229_0.79_synergy_trend_col.svg'% OUTPUT_DIR)

#### Fig. S1c

In [ ]:
kw_df = pd.DataFrame(auc_syninfo_d.keys())
fig2,ax2a = plt.subplots(2,2,dpi=180,sharey=True,sharex=True,figsize=(6,4))
objval_clr_d = {'0.00':'C0','0.79':'C1','1.57':'C2'}
objval_str_d = {'0.00':'Behavior','0.79':'Combined','1.57':'Growth'}
objval_num_d = {'0.00':0,'0.79':1,'1.57':2}
kk_num_d = {'linear':0,'nonadd':1}
auc_syntstat_d = {}
for ii,CL in enumerate(kw_df.iloc[:,0].unique()):    
    ROW,COL = (ii//2),(ii%2)
    ax2 = ax2a[ROW,COL]
    ax2.set_title(CL,size=8)
    if COL ==0:
        ax2.set_ylabel('Synergy AUC',size=8)
        ax2.set_ylim(0,1)
        plt.setp(ax2.get_yticklabels(),size=6)
    clr_dat_d = defaultdict(list)
    for (CLz,objval,kk) in kw_df[kw_df.iloc[:,0]==CL].values:
        clr = objval_clr_d[objval]
        pos=2*objval_num_d[objval]+kk_num_d[kk]
        clr_dat_d[clr].append((auc_syninfo_d[(CL,objval,kk)],pos,kk))
    for (CLx,objval1,kk1),(CLy,objval2,kk2) in combs(kw_df[kw_df.iloc[:,0]==CL].values,2):
        auc_l1 = auc_syninfo_d[(CL,objval1,kk1)]
        auc_l2 = auc_syninfo_d[(CL,objval2,kk2)]
        tstat,pval = ranksums(auc_l1,auc_l2)
        auc_syntstat_d[(CL,objval1,kk1,objval2,kk2,'rstat')]=tstat
        auc_syntstat_d[(CL,objval1,kk1,objval2,kk2,'pval')]=pval        
    clr_dat_d = dict(clr_dat_d)
    clr_l = []
    box_l = []
    for clr,dat in clr_dat_d.items():
        data_l,pos_l,lbl_l = zip(*dat)
        lbl_l = ['' for elt in lbl_l] if ROW==0 else lbl_l
        box = ax2.boxplot(data_l,positions=pos_l,labels=lbl_l,widths=0.6,patch_artist=True,
                         notch=False,sym='%s.' % clr,boxprops={'color':clr,'alpha':0.5},
                         whiskerprops={'color':'k'}, capprops={'color':'k'},
                         medianprops={'color':'C3'},flierprops={'alpha':0.5,'markeredgecolor':None})
        box_l.append(box)
        clr_l.append(clr)
    for clr,bplot in zip(clr_l,box_l):
        sclr = [box.set_color(clr) for box in bplot['boxes']]
        sal = [box.set_alpha(0.5) for box in bplot['boxes']]
    ax2.axhline(0.5,ls='--',color='C7')
    if ROW==1:
        plt.setp(ax2.get_xticklabels(),rotation=90,size=6)
fig2.savefig('%s/ranked_auc_synergy.svg'% OUTPUT_DIR)
auc_syntstat_ser=pd.Series(auc_syntstat_d)
auc_syntstat_ser = auc_syntstat_ser.unstack(0)
auc_syntstat_ser.to_excel('%s/ranked_synergy_auc_tstats.xlsx'% OUTPUT_DIR)


### Fig. S2

In [ ]:
fig,ax_l = plt.subplots(1,4,dpi=180,figsize=(6.11,1.82),sharey=True)
colkw ='loewe_12'
clr_d = defaultdict(dict)
objval_clr_d = {'0.00':'C0','0.79':'C1','1.57':'C2'}
torb_alpha_d = {'Top':0.9,'Bottom':0.5}
num_d = {'0.00':0,'0.79':2,'1.57':4,'additive':0,'nonadditive':1}
NP = 1000
tb_spc = 0.5
for ar,CL in enumerate(['CH-157MN','GBM43','IOMM-LEE','LN-229']):
    print(CL)
    dat_d = defaultdict(dict)
    #tot_l = [5,22.5,40]
    lbl_d = defaultdict(dict)
    pos_d = defaultdict(dict)
    pos_ind = 0
    for col in comb_pred.columns:
        data_d = cl_drg_data_d[CL]
        DATA = pd.DataFrame(data_d).T
        DATA.index = pd.MultiIndex.from_tuples([k if isinstance(k,tuple) else (k,'dmso') for k in DATA.index])
        sum_pred_ranks = comb_pred.loc[:,col].fillna(0).rank(pct=True,ascending=False)
        for kw in ['Top','Bottom']:
            if kw == 'Top':
                tot_sp = [elt for elt in ld_pairs if sum_pred_ranks.loc[elt]<BP_THR]
            else:
                tot_sp = [elt for elt in ld_pairs if sum_pred_ranks.loc[elt]>=BP_THR]

            sel_ld_keys = [tuple([val+'_ld' for val in elt]) for elt in tot_sp]
            rnd_avg_l = []
            if NP>len(sel_ld_keys):
                NP=len(sel_ld_keys)
            dat_d[col[1]][(kw,col[0])] = (1-DATA.loc[sel_ld_keys].mean(axis=1))/totaldosage.loc[sel_ld_keys]
            pos_ind = num_d[col[1]]+num_d[col[0]]
            pos_d[col[1]][(kw,col[0])] = pos_ind
            if kw=='Top':
                lbl_d[col[1]][(kw,col[0])] = ('%s' % (col[0]))
    dat_d = dict(dat_d)
    pos_d = dict(pos_d)
    lbl_d = dict(lbl_d)
    xticks=[]
    xticklabels=[]
    for objval,method_dat_d in dat_d.items():
        clr = objval_clr_d[objval]
        for (torb,aorna),data_list in method_dat_d.items():
            if torb=='Bottom':
                xpos = pos_d[objval][(torb,aorna)] + tb_spc
            else:
                xpos = pos_d[objval][(torb,aorna)]
            if torb=='Top':
                xticks.append(pos_d[objval][(torb,aorna)] + tb_spc/2.)
                xticklabels.append(lbl_d[objval][(torb,aorna)])
            boxes0 = ax_l[ar].boxplot(data_list,labels=[torb],widths=[0.3],
                                      sym='%s.' % clr, positions=[xpos],
                                      patch_artist=True, notch=False,
                                      boxprops={'color':clr,'alpha':torb_alpha_d[torb]},
                                      whiskerprops={'color':'k'},capprops={'color':'k'},medianprops={'color':'C3'},
                                      flierprops={'alpha':0.5,'markeredgecolor':None})
            sclr = [boxes0['boxes'][xx].set_color(clr) for xx in range(len(boxes0['boxes']))]
            sal = [boxes0['boxes'][xx].set_alpha(torb_alpha_d[torb]) for xx in range(len(boxes0['boxes']))]
    ax_l[ar].set_title(CL,size=8)
    ax_l[ar].set_xticks(xticks)
    ax_l[ar].set_xticklabels(xticklabels,size=6,rotation=90)
    plt.setp(ax_l[ar].get_xticklabels(),size=6,rotation=90)
ax_l[0].set_ylabel(r'Viability reduction per $\mu$M',size=8)
plt.setp(ax_l[0].get_yticklabels(),size=6)
ax_l[0].set_yscale('log')
    

fig.savefig('%s/figS2_low_dosage_viability.svg'% OUTPUT_DIR)

### Fig. S3

#### Fig. S3 statistics

In [ ]:
## code to calculate the synergy statistics
op_d = {}
for CL,synstat_df in cl_synstat_d.items():
    ## synergy rates above and below the viability thresholds
    high_thr = (synstat_df.loc[:,colkw]/synstat_df.v_12>1)[synstat_df.loc[:,colkw]>BP_THR].mean()
    low_thr = (synstat_df.loc[:,colkw]/synstat_df.v_12>1)[synstat_df.loc[:,colkw]<BP_THR].mean()
    print(CL,high_thr,low_thr)
    sel_ld_d = {}
    sel_ncld_d = {}
    unsel_ld_d = {}    
    for d1,d2 in ld_pairs:
        d1p = d1+'_ld'
        d2p = d2+'_ld'
        if (d1,d2) in sel_inds_d.keys():
            sel_ld_d[('observed',d1,d2)] = (synstat_df.loc[(d1p,d2p),'v_12'] < synstat_df.loc[(d1p,d2p),colkw])
            sel_ld_d[('expected',d1,d2)] = low_thr if synstat_df.loc[(d1p,d2p),colkw]<BP_THR else high_thr
        elif (d2,d1) in sel_inds_d.keys():
            sel_ld_d[('observed',d1,d2)] = (synstat_df.loc[(d1p,d2p),'v_12'] < synstat_df.loc[(d1p,d2p),colkw])
            sel_ld_d[('expected',d1,d2)] = low_thr if synstat_df.loc[(d1p,d2p),colkw]<BP_THR else high_thr
        else:
            unsel_ld_d[('observed',d1,d2)] = (synstat_df.loc[(d1p,d2p),'v_12'] < synstat_df.loc[(d1p,d2p),colkw])
            unsel_ld_d[('expected',d1,d2)] = low_thr if synstat_df.loc[(d1p,d2p),colkw]<BP_THR else high_thr
    ## estimate the synergy ratio statistics (low dosage)
    for lbl,sel_d in zip(['top_ld','unsel_ld'],[sel_ld_d,unsel_ld_d]):
        sel_df = pd.Series(sel_d).unstack(level=0).astype(float)
        newindex = [(e1+'_ld',e2+'_ld') for e1,e2 in sel_df.index]
        for knd,val in sel_df.mean().items():
            op_d[(CL,lbl,knd)]=val
        op_d[(CL,lbl,'ratio')]=op_d[(CL,lbl,'observed')]/op_d[(CL,lbl,'expected')]
        obs_l = []
        exp_l = []
        rat_l = []
        ## bootstrap the synergy ratio (low dosage)
        for mm in range(100):
            rac = synstat_df.loc[newindex,'v_12']+synstat_df.loc[newindex,'v_12_std']*np.random.randn(sel_df.shape[0])
            rloewe_d = {}
            for (kk0,kk1) in sel_df.index:
                rloewe_d[(kk0+'_ld',kk1+'_ld')] = 1/(10/(myic50_df.loc[kk0,'mu']+myic50_df.loc[kk0,'sig']*np.random.randn())+
                                        10/(myic50_df.loc[kk1,'mu']+myic50_df.loc[kk1,'sig']*np.random.randn())+1)            
            rloewe = pd.Series(rloewe_d)
            rexp = ((rloewe<BP_THR)*low_thr+(rloewe>BP_THR)*high_thr)
            robs =(rac<rloewe).mean()
            obs_l.append(robs)
            exp_l.append(rexp.mean())
            rat_l.append(robs/rexp.mean())
        op_d[(CL,lbl,'observed_std')]=np.std(obs_l)
        op_d[(CL,lbl,'expected_std')]=np.std(exp_l)
        op_d[(CL,lbl,'ratio_std')]=np.std(rat_l)
        
    ## estimate the synergy ratio statistics
    for lbl,kk_l in lbl_kw_d.items():
        act_syn = (synstat_df.loc[kk_l,'v_12'] < synstat_df.loc[kk_l,colkw]).mean()
        bl_syn = (synstat_df.loc[kk_l,colkw]<BP_THR)*low_thr+(synstat_df.loc[kk_l,colkw]>BP_THR)*high_thr
        op_d[(CL,lbl,'observed')]=act_syn
        op_d[(CL,lbl,'expected')]=bl_syn.mean()
        op_d[(CL,lbl,'ratio')]=act_syn/bl_syn.mean()
        obs_l = []
        exp_l = []
        rat_l = []
        ## bootstrap the synergy ratio
        for mm in range(100):
            rac = synstat_df.loc[kk_l,'v_12']+synstat_df.loc[kk_l,'v_12_std']*np.random.randn(len(kk_l))
            rloewe_d = {}
            for (kk0,kk1) in kk_l:
                rloewe_d[(kk0,kk1)] = 1/(10/(myic50_df.loc[kk0,'mu']+myic50_df.loc[kk0,'sig']*np.random.randn())+
                                        10/(myic50_df.loc[kk1,'mu']+myic50_df.loc[kk1,'sig']*np.random.randn())+1)            
            rloewe = pd.Series(rloewe_d)
            rexp = ((rloewe<BP_THR)*low_thr+(rloewe>BP_THR)*high_thr)
            robs =(rac<rloewe).mean()
            obs_l.append(robs)
            exp_l.append(rexp.mean())
            rat_l.append(robs/rexp.mean())
        op_d[(CL,lbl,'observed_std')]=np.std(obs_l)
        op_d[(CL,lbl,'expected_std')]=np.std(exp_l)
        op_d[(CL,lbl,'ratio_std')]=np.std(rat_l)

gross_synergy_stats = pd.Series(op_d).unstack()
gross_synergy_stats.to_excel('%s/gross_synergy_stats.xlsx'% OUTPUT_DIR)

#### Fig. S3 figure

In [ ]:
comp_kwp_d = {
    'nonadd_v_add':(('nonadditive', 'comb', 'top'),('additive', 'comb', 'top')),
    'bhv_nonadd_v_add':(('nonadditive', '0.00', 'top'),('additive', '0.00', 'top')),
    'comb_nonadd_v_add':(('nonadditive', '0.79', 'top'),('additive', '0.79', 'top')),
    'growth_nonadd_v_add':(('nonadditive', '1.57', 'top'),('additive', '1.57', 'top')),
    'growth_tvb_bhv':(('comb', '0.00', 'top'),('comb', '0.79', 'top'),('comb', '1.57', 'top'),('comb', '1.57', 'bot')),
    'add_bhv':(('additive', '0.00', 'top'),('additive', '0.79', 'top'),('additive', '1.57', 'top')),
    'nonadd_bhv':(('nonadditive', '0.00', 'top'),('nonadditive', '0.79', 'top'),('nonadditive', '1.57', 'top'))
}


comp_labels_d = {
    'nonadd_v_add':(('Nonadditive','C7'),('Additive', 'C7')),
    'bhv_nonadd_v_add':(('Nonadditive','C0'),('Additive', 'C0')),
    'comb_nonadd_v_add':(('Nonadditive', 'C1'),('Additive', 'C1')),
    'growth_nonadd_v_add':(('Nonadditive', 'C2'),('Additive', 'C2')),
    'growth_tvb_bhv':(('Behavior','C0'),('Combined', 'C1'),('Growth', 'C2'),('Non-growth','C3')),
    'nonadd_bhv':(('Behavior','C0'),('Combined', 'C1'),('Growth', 'C2')),
    'add_bhv':(('Behavior','C0'),('Combined', 'C1'),('Growth', 'C2')),
}


In [ ]:
plt.rcParams['svg.fonttype']='none'
stats_d = {}
fig,ax_l = plt.subplots(4,2,dpi=180,figsize=(6,6),sharex=True)
axi=0
for CL,drg_data_d in cl_drg_data_d.items():
    ## compare the drug viability data
    print(CL)
    ax=ax_l[axi,0]
    axi +=1
    DATA = pd.DataFrame(drg_data_d).T
    DATA.index = pd.MultiIndex.from_tuples([k if isinstance(k,tuple) else (k,'dmso') for k in DATA.index])
    pos_ind = 0
    clr_data_d = defaultdict(list)
    for comp_nm, inds_l in comp_kwp_d.items():
        grp_l = []
        for inds in inds_l:
            MI = pd.DataFrame(index_dict[inds].tolist()).replace('brd-k63750851','mycophenolic acid').set_index([0,1]).index
            msng = MI.difference(DATA.index)
            if msng.shape[0]>0:
                add = MI.swaplevel().intersection(DATA.index)
                MI = MI.difference(msng).union(add)
            grp = DATA.loc[MI]
            grp_l.append(grp)
        if comp_nm.endswith('nonadd_v_add'):            
            grpA,grpB = grp_l
            (lbl1,clr1),(lbl2,clr2) = comp_labels_d[comp_nm]
            clr_data_d[clr1].append((grpA.mean(axis=1),lbl1,pos_ind))
            clr_data_d[clr2].append((grpB.mean(axis=1),lbl2,pos_ind+1))
            rss,rspv = ranksums(grpA.mean(axis=1),grpB.mean(axis=1))
            tts,ttpv = ttest_ind(grpA.mean(axis=1),grpB.mean(axis=1))
            stats_d[(CL,comp_nm,'rs_stat')] = rss
            stats_d[(CL,comp_nm,'rs_pval')] = rspv
            stats_d[(CL,comp_nm,'tt_stat')] = tts
            stats_d[(CL,comp_nm,'tt_pval')] = ttpv
            pos_ind+=2
        else:
            #pos_ind+=1
            lbl_grp_d ={}
            for zz,grp in enumerate(grp_l):
                lbl,clr = comp_labels_d[comp_nm][zz]
                lbl_grp_d[lbl]=grp
                if comp_nm=='growth_tvb_bhv':
                    clr_data_d[clr].append((grp.mean(axis=1),lbl,pos_ind))
                    pos_ind+=1
            for (lbl1,grp1),(lbl2,grp2) in combs(lbl_grp_d.items(),2):
                rss,rspv = ranksums(grpA.mean(axis=1),grpB.mean(axis=1))
                tts,ttpv = ttest_ind(grpA.mean(axis=1),grpB.mean(axis=1))
                str_l=[comp_nm,lbl1,lbl2]
                stats_d[(CL,'_'.join(str_l),'rs_stat')] = rss
                stats_d[(CL,'_'.join(str_l),'rs_pval')] = rspv
                stats_d[(CL,'_'.join(str_l),'tt_stat')] = tts
                stats_d[(CL,'_'.join(str_l),'tt_pval')] = ttpv
            
    clr_data_d = dict(clr_data_d)
    bplot_l = []
    clr_l = []
    for clr,data_l in clr_data_d.items():
        boxdat_l,lbl_l,pos_l = zip(*data_l)
        box = ax.boxplot(boxdat_l,positions=pos_l,labels=lbl_l,widths=0.6,patch_artist=True,notch=False,sym='%s.' % clr,
                         boxprops={'color':clr,'alpha':0.5},whiskerprops={'color':'k'}, capprops={'color':'k'},
                         medianprops={'color':'C3'},flierprops={'alpha':0.5,'markeredgecolor':None})
        bplot_l.append(box)
        clr_l.append(clr)
    ax.fill_betweenx([0,1.5],x1=1.5,x2=7.5,color='C7',alpha=0.2)
    #ax.fill_betweenx([0,1.05],x1=17,x2=20.5,color='C7',alpha=0.2)
    for clr,bplot in zip(clr_l,bplot_l):
        sclr = [box.set_color(clr) for box in bplot['boxes']]
        sal = [box.set_alpha(0.5) for box in bplot['boxes']]
    ax.set_title(CL,size=8)
    ax.set_ylabel('Viability',size=8)
    plt.setp(ax.get_yticklabels(),size=6)
    ax.set_yscale('log')
    ax.set_ylim(4.5e-4,1.5)
plt.setp(ax.get_xticklabels(),rotation=90,size=6)

print('')
## plot the synergy data
axi=0
colkw = 'loewe_12'
cl_sstats_d = {}
for CL,synstat_df in cl_synstat_d.items():
    print(CL)
    pos_ind=0
    sstats_d = {}
    clr_data_d = defaultdict(dict)
    high_thr = (synstat_df.loc[:,colkw]/synstat_df.v_12>1)[synstat_df.loc[:,colkw]>BP_THR].mean()
    low_thr = (synstat_df.loc[:,colkw]/synstat_df.v_12>1)[synstat_df.loc[:,colkw]<BP_THR].mean()

    ## need to calculate the expected rate of synergy.
    for comp_nm, inds_l in comp_kwp_d.items():
    #     if comp_nm in ['nonadd_v_add','comb_v_behave','bhv_nonadd_v_add','comb_nonadd_v_add',
    #                    'growth_nonadd_v_add','nonadd_bhv_v_growth','add_bhv_v_growth','nonadd_bhv_v_comb']:
    #         continue
        stats_l = []
        for inds in inds_l:
            MI = pd.DataFrame(index_dict[inds].tolist()).replace('brd-k63750851','mycophenolic acid').set_index([0,1]).index
            msng = MI.difference(synstat_df.index)
            if msng.shape[0]>0:
                add = MI.swaplevel().intersection(synstat_df.index)
                MI = MI.difference(msng).union(add)
            grp = synstat_df.loc[MI]
            obs = (grp.v_12 < grp.loc[:,colkw]) ## rate of synergy
            exp = (grp.loc[:,colkw]<BP_THR).mean() * low_thr + (grp.loc[:,colkw]>BP_THR).mean() * high_thr
            rat = (obs/exp).mean()
            for mm in range(100):
                rac = grp.v_12+grp.v_12_std*np.random.randn(grp.shape[0])
                rloewe_d = {}
                for (kk0,kk1) in grp.index:
                    rloewe_d[(kk0,kk1)] = 1/(10/(myic50_df.loc[kk0,'mu']+myic50_df.loc[kk0,'sig']*np.random.randn())+
                                            10/(myic50_df.loc[kk1,'mu']+myic50_df.loc[kk1,'sig']*np.random.randn())+1)            
                rloewe = pd.Series(rloewe_d)
                rexp = ((rloewe<BP_THR)*low_thr+(rloewe>BP_THR)*high_thr)
                robs =(rac<rloewe).mean()
                obs_l.append(robs)
                exp_l.append(rexp.mean())
                rat_l.append(robs/rexp.mean())
            obs_std=np.std(obs_l)
            exp_std=np.std(exp_l)
            rat_std=np.std(rat_l)
            ## estimate the obs/exp/rat standard deviations
            stats_l.append((obs,exp,rat,obs_std,exp_std,rat_std,grp.shape[0]))
        if comp_nm.endswith('nonadd_v_add'):            
            stats1,stats2 = stats_l
            (lbl1,clr1),(lbl2,clr2) = comp_labels_d[comp_nm]
            obs1,exp1,rat1,obs_std1,exp_std1,rat_std1,nobs1=stats1
            obs2,exp2,rat2,obs_std2,exp_std2,rat_std2,nobs2=stats2
            ## do we want 
            tts,ttpv = ttest_ind_from_stats(rat1,rat_std1,nobs1,rat2,rat_std2,nobs2)
            clr_data_d[clr1][lbl1] = (rat1,rat_std1/np.sqrt(nobs1),lbl,pos_ind)
            clr_data_d[clr2][lbl2] = (rat2,rat_std2/np.sqrt(nobs2),lbl,pos_ind+1)
            sstats_d[(CL,comp_nm,'tt_stat')] = tts
            sstats_d[(CL,comp_nm,'tt_pval')] = ttpv
            pos_ind+=2
        else:
            #pos_ind+=1
            lbl_grp_d ={}
            for zz,grp in enumerate(stats_l):
                lbl,clr = comp_labels_d[comp_nm][zz]
                lbl_grp_d[lbl]=grp
                if comp_nm=='growth_tvb_bhv':
                    clr_data_d[clr][lbl] = (grp[2],grp[5]/np.sqrt(grp[6]),lbl,pos_ind)
                    pos_ind+=1
            for (lbl1,grp1),(lbl2,grp2) in combs(lbl_grp_d.items(),2):
                obs1,exp1,rat1,obs_std1,exp_std1,rat_std1,nobs1=grp1
                obs2,exp2,rat2,obs_std2,exp_std2,rat_std2,nobs2=grp2
                ## do we want 
                tts,ttpv = ttest_ind_from_stats(rat1,rat_std1,nobs1,rat2,rat_std2,nobs2)            
                str_l=[comp_nm,lbl1,lbl2]
                sstats_d[(CL,'_'.join(str_l),'tt_stat')] = tts
                sstats_d[(CL,'_'.join(str_l),'tt_pval')] = ttpv
    clr_data_d = dict(clr_data_d)
    xtls = []
    xinds = []
    ax=ax_l[axi,1]
    axi +=1
    for clr,data in clr_data_d.items():
        data_df = pd.DataFrame(data)
        ax.errorbar(data_df.iloc[3],data_df.iloc[0],yerr=data_df.iloc[1],fmt='%s.' % clr)
        xinds.extend(data_df.iloc[3].values.tolist())
        for ind,col in data_df.items():
            xtls.append('%s' % (ind))
    srt_xtls,srt_ticks = list(zip(*sorted(zip(xtls,xinds),key=lambda e: e[1])))
    ax.fill_betweenx(y=[0,3],x1=1.5,x2=7.5,color='C7',alpha=0.2)
    ax.set_ylim(0.3,3)
    ax.set_yscale('log',base=2)
    ax.set_title(CL,size=8,family='sans')
    ax.set_ylabel('Synergy ratio',size=8,family='sans')
    ax.axhline(1,ls='--',color='C7')
    cl_sstats_d[CL] = sstats_d
    plt.setp(ax.get_yticklabels(),size=6,family='sans')
ax.set_xticks(srt_ticks)
ax.set_xticklabels(srt_xtls,size=6,rotation=90,family='sans')
fig.savefig('%s/figS3_exclusive_pairs_combined.svg' % OUTPUT_DIR)
print('')
cl_sstats_df = pd.DataFrame(cl_sstats_d).stack().droplevel(0).unstack([-1,-2])
cl_sstats_df.to_excel('%s/gross_synergy_pvals.xlsx' % OUTPUT_DIR)

### Fig. S5 (dose-response curves)

In [ ]:
for CL in ['IOMM-Lee','CH157']:
    obs1210 = pd.read_excel('%s/2021-12-10-CTG-CH157-Lee.xlsx' % INPUT_DIR,engine='openpyxl',sheet_name='%s Torin Calc' % CL,usecols='AA:AL',index_col=0,header=1,nrows=11)
    #obs1210= obs1210.iloc[:,1:12]
    obs1210.columns = ['1uM','0.5uM','0.25uM','0.125uM','0.063uM','0.031uM','0.016uM','0.008uM','0.004uM','0.002uM','0uM']
    obs1210.index = [elt.split(' ')[0] for elt in obs1210.index]
    obs1210.index.name='Digitoxin'
    obs1210.columns.name='Torin 2'
    pred1210 = pd.read_excel('%s/2021-12-10-CTG-CH157-Lee.xlsx' % INPUT_DIR,engine='openpyxl',sheet_name='%s Torin Calc' % CL,usecols='AA:AL',index_col=0,header=14,nrows=11)
    #pred1210= pred1210.iloc[:,1:12]
    pred1210.columns = ['1uM','0.5uM','0.25uM','0.125uM','0.063uM','0.031uM','0.016uM','0.008uM','0.004uM','0.002uM','0uM']
    pred1210.index = [elt.split(' ')[0] for elt in pred1210.index]
    pred1210.index.name='Digitoxin'
    pred1210.columns.name='Torin 2'

    figsize_in = (7.5,1.83)

    lblpad = 0.05
    cbpad = 0.005
    cblbl = 0.075
    cbwidth = 0.015
    pwidth = 1/3 - (lblpad + cbpad + cblbl + cbwidth)

    toppad = .025
    pheight = .75 
    botpad = .225

    fig = plt.figure(figsize=figsize_in,dpi=180)
    ax1 = fig.add_axes([lblpad,botpad,pwidth,pheight])
    ax2 = fig.add_axes([lblpad+(lblpad+pwidth+cbpad+cbwidth+cblbl),botpad,pwidth,pheight])
    ax3 = fig.add_axes([lblpad+2*(lblpad+pwidth+cbpad+cbwidth+cblbl),botpad,pwidth,pheight])

    cbax1 = fig.add_axes([lblpad+pwidth+cbpad,botpad,cbwidth,pheight])
    cbax2 = fig.add_axes([lblpad+pwidth+cbpad+(lblpad+pwidth+cbpad+cbwidth+cblbl),botpad,cbwidth,pheight])
    cbax3 = fig.add_axes([lblpad+pwidth+cbpad+2*(lblpad+pwidth+cbpad+cbwidth+cblbl),botpad,cbwidth,pheight])
    vcm = plt.get_cmap('viridis')

    LN = LogNorm(vmin=1e-2, vmax=10,clip=True)
    Ndose = obs1210.shape[0]
    obj_l = []
    lvl_l = []
    lw_l = []
    ls_l = []
    clr_l = []
    alpha=0.1
    for ii in range(Ndose-1,-1,-1):
        numcols = [float(nm[:-2])+1e-3 for nm in obs1210.columns][::-1]
        CC = float(obs1210.index[ii][:-2])+1e-4
        obj =ax1.plot(numcols,1-obs1210.iloc[ii,::-1],color=vcm(LN(CC)),ls='--',marker='o',alpha=0.5,label='%.2e' % CC)
        xdat,ydat = obj[0].get_data()
        lw_l.append(obj[0].get_lw())
        ls_l.append(obj[0].get_ls())
        clr_l.append(obj[0].get_color())
    #     kw_conv_d = {'edgecolor':obj[0].get_color(),
    #                  'figure':obj[0].get_figure(),
    #                  'linestyle':obj[0].get_ls(),
    #                  'linewidth':obj[0].get_lw(),
    #                  'fill':False,
    #                 }
        #pg = mpl.patches.Polygon(np.c_[xdat[:3],ydat[:3]],closed=False, **kw_conv_d)    
        obj_l.append([np.c_[xdat[:3],ydat[:3]].tolist()])
        lvl_l.append(CC)
    CS = mpl.contour.ContourSet(ax1,lvl_l,obj_l,linewidths=lw_l,linestyles=ls_l,colors=clr_l,norm=LN)
    cb1 = fig.colorbar(CS,cax=cbax1,format=mpl.ticker.LogFormatter(2,labelOnlyBase=False))
    ax1.set_xscale('log')
    plt.setp(ax1.get_xticklabels(),size=6)
    plt.setp(ax1.get_yticklabels(),size=6)
    ax1.set_ylabel('Viability',size=6)
    ax1.set_xlabel(obs1210.columns.name + r' ($\mu$M)',size=6)
    cb1.set_label(obs1210.index.name + r' ($\mu$M)',size=6)
    plt.setp(cb1.ax.yaxis.get_ticklabels(),size=6)

    LN2 = LogNorm(vmin=1e-3, vmax=1,clip=True)
    Ndose = obs1210.shape[1]
    obj_l = []
    lvl_l = []
    lw_l = []
    ls_l = []
    clr_l = []
    alpha=0.1
    for ii in range(Ndose-1,-1,-1):
        numcols = [float(nm[:-2])+1e-2 for nm in obs1210.index][::-1]
        CC = float(obs1210.columns[ii][:-2])+1e-4
        obj =ax2.plot(numcols,1-obs1210.iloc[::-1,ii],color=vcm(LN2(CC)),ls='--',marker='o',alpha=0.5,label='%.2e' % CC)
        xdat,ydat = obj[0].get_data()
        lw_l.append(obj[0].get_lw())
        ls_l.append(obj[0].get_ls())
        clr_l.append(obj[0].get_color())
    #     kw_conv_d = {'edgecolor':obj[0].get_color(),
    #                  'figure':obj[0].get_figure(),
    #                  'linestyle':obj[0].get_ls(),
    #                  'linewidth':obj[0].get_lw(),
    #                  'fill':False,
    #                 }
        #pg = mpl.patches.Polygon(np.c_[xdat[:3],ydat[:3]],closed=False, **kw_conv_d)    
        obj_l.append([np.c_[xdat[:3],ydat[:3]].tolist()])
        lvl_l.append(CC)
    CS = mpl.contour.ContourSet(ax2,lvl_l,obj_l,linewidths=lw_l,linestyles=ls_l,colors=clr_l,norm=LN)
    cb2 = fig.colorbar(CS,cax=cbax2,format=mpl.ticker.LogFormatter(2,labelOnlyBase=False))
    ax2.set_xscale('log')
    ax2.set_ylabel('Viability',size=6)
    plt.setp(ax2.get_xticklabels(),size=6)
    plt.setp(ax2.get_yticklabels(),size=6)
    ax2.set_xlabel(obs1210.index.name + r' ($\mu$M)',size=6)
    cb2.set_label(obs1210.columns.name + r' ($\mu$M)',size=6)
    plt.setp(cb2.ax.yaxis.get_ticklabels(),size=6)

    
    mycmap = plt.get_cmap('bwr')
    img = ax3.imshow((obs1210-pred1210).iloc[4:10,4:10].iloc[::-1,::-1]*100,origin='lower',cmap=mycmap,norm=CenteredNorm())
    ax3.set_xticks(np.arange(6))
    ax3.set_yticks(np.arange(6))
    ax3.set_xticklabels([ '%.3f' % float(col[:-2]) for col in  obs1210.columns[4:10]][::-1],size=6,horizontalalignment='right',rotation=30)
    ax3.set_yticklabels([ '%.2f' % float(col[:-2]) for col in  obs1210.index[4:10]][::-1],size=6)
    ax3.set_xlabel(r'Torin 2 ($\mu$M)',size=6)
    ax3.set_ylabel(r'Digitoxin ($\mu$M)',size=6)
    cb3 = fig.colorbar(img,cax=cbax3)
    cb3.set_label('Bliss synergy (%)',size=6)
    plt.setp(cb3.ax.yaxis.get_ticklabels(),size=6)
    fig.savefig('%s/figS5_dose_response_%s_digitoxin_torin2.svg' % (OUTPUT_DIR,CL))

### Fig. 4

#### Fig. 4a

In [ ]:
stacked_dd = defaultdict(int) ## container for the drug counts
LBL_D = {'Unselected':'Unselected',
         'Low-dosage':'Low-dosage',
         'Anti-selected':'Anti-selected',
         'Non-cancer':'Non-cancer',
         'Top selected (brown)':'Top (consensus)',
         'Top selected (purple)':'NA',
         'Other':'NA2'}
OTHER_GRPS = {'Additive average (gray)', 'Behavior additive (blue)', 'Behavior average (blue)', 'Behavior nonadditive (blue)',
 'Combined additive (orange)', 'Combined average (orange)', 'Combined nonadditive (orange)', 'Growth additive (green)',
 'Growth average (green)', 'Growth nonadditive (green)', 'Non-growth average (red)', 'Nonadditive average (gray)'}
SEL_GRPS = ['Unselected (yellow)','Low dosage (cyan)','Anti-selected (pink)','Non-cancer (brown)','Top selected (brown)','Top selected (purple)']
for grpnm,LBL in LBL_D.items():
    if grpnm in ['Top selected (purple)']:
        ## need to distinguish between drugs in 'Top selected (brown)' group and not
        UN_NM = test_grps[test_grps.group==grpnm].drug1_name.unique()
        UN_NM1 = test_grps[test_grps.group=='Top selected (brown)'].drug1_name.unique()
        UN_NM2 = test_grps[test_grps.group=='Top selected (brown)'].drug2_name.unique()
        stacked_dd[('Top (specific)','Single drugs')] = len(set(UN_NM) - (set(UN_NM1) | set(UN_NM2))) ## number of single drugs in 'Top (specific)'
        stacked_dd[('Top (consensus)','Single drugs')] = len(set(UN_NM) & (set(UN_NM1) | set(UN_NM2))) ## number of single drugs in 'Top (consensus)'
    else:
        if grpnm in ['Unselected','Low-dosage','Anti-selected','Non-cancer','Top selected (brown)']:
            sel = test_grps[test_grps.group==grpnm] 
            LBL = LBL_D[grpnm] 
        else:
            LBL = 'Top (specific)'
            sel = test_grps[test_grps.group.isin(OTHER_GRPS)]
        sel_sng = sel[sel.drug2_name.isna()]
        sel_pair = sel[~sel.drug2_name.isna()]
        ## save as part of drug pairs
        stacked_dd[(LBL,'Drug pairs')] = sel_pair.shape[0] 
        if sel_sng.shape[0]>0:
            ## save as part of single drugs
            stacked_dd[(LBL,'Single drugs')] = sel_sng.shape[0] 
    
stacked_data_df = pd.Series(dict(stacked_dd)).unstack()


# purple, yellow, red, light blue, pink
clr_l = ['C3','C3','C4','#EDB120','#4DBEEE','#E377c2']
grplbls = ['Top (consensus)','Top (specific)','Non-cancer','Anti-selected','Unselected','Low-dosage']
fig,ax = plt.subplots(1,1,dpi=180,frameon=False,figsize=(2,3))
for ii,col in enumerate(['Drug pairs','Single drugs']):
    ll = 0 
    for clr,lbl in zip(clr_l,grplbls):
        hh = stacked_data_df.loc[lbl,col]
        ax.bar([ii],height=[hh],bottom=[ll],label='%s (%d)'%(lbl,hh),color=clr,alpha=0.8)
        ll+=hh
ax.set_xlim(-0.5,1.5)
ax.legend(frameon=False,prop={'size':6},ncol=2)
ax.set_xticks([0,1])
ax.set_xticklabels(['Drug pairs','Single drugs'],size=6, horizontalalignment='center')
ax.set_ylabel('Number',size=8)
plt.setp(ax.get_yticklabels(),size=6)
#ax.set_frameon(False)
#fig.set_axes(False)
#fig.savefig('%s/fig4a_drug_selection_counts.svg'% OUTPUT_DIR)

#### Fig. 4bc

In [ ]:
top_sel_consensus = list(set(index_dict[('all','all','top')])-set(shared_l))
top_sel_specific = set([])
for k,v in index_dict.items():
    if k == ('all', 'all', 'top'):
        continue
    else:
        top_sel_specific |= (set(v) - set(shared_l))
        
top_sel_specific = list(top_sel_specific)
top_sel_specific2 = []
for (x,y) in top_sel_specific:
    if x=='brd-k63750851':
        xp = 'mycophenolic acid'
    elif x=='hydrocortisone':
        xp = 'cortisol'
    else:
        xp = x
    if y=='tretinoin':
        yp = 'retinoic acid'
    else:
        yp = y
    top_sel_specific2.append((xp,yp))
        ## cycle through and replace...
top_sel_specific=top_sel_specific2
grplbls = ['Top (consensus)','Top (model-specific)','Non-cancer','Anti-selected',
           'Unselected','Low-dosage']
grplbl_d = {'top':'Top (consensus)', 'top_spec':'Top (model-specific)' ,
            'noncanc':'Non-cancer','anti':'Anti-selected','random':'Unselected'}
drggrps = [top_sel_consensus,top_sel_specific,noncanc_l,antisel_pairs,
           nonsel_pairs,ld_pairs]

axi=0
## make the boxplots
fig,ax_l = plt.subplots(2,4,dpi=180,figsize=(4,3),sharex='col',sharey='row')
ovrstats_d = {}
for CL,drg_data_d in cl_drg_data_d.items():
    print(CL)
    ax=ax_l[0,axi]
    axi +=1
    DATA = pd.DataFrame(drg_data_d).T
    DATA.index = pd.MultiIndex.from_tuples([k if isinstance(k,tuple) else (k,'dmso') for k in DATA.index])
    pos=0    
    for clr,lbl,grp in zip(clr_l,grplbls,drggrps):
        if lbl in ['Low-dosage','Top (model-specific)']:
            continue
        grpdat = DATA.loc[grp].mean(axis=1)
        boxes = ax.boxplot([grpdat],labels=[lbl],positions=[pos],patch_artist=True,notch=False,
                           boxprops={'color':clr,'alpha':1},whiskerprops={'color':'k'},
                           widths=0.6,capprops={'color':'k'},
                           medianprops={'color':'C3'},flierprops={'marker':'.','markerfacecolor':clr,'alpha':0.5,'markeredgecolor':'none'})

        sclr = boxes['boxes'][0].set_color(clr)
        sal = boxes['boxes'][0].set_alpha(0.8)
        pos+=1
    for (lbl1,grp1),(lbl2,grp2) in combs(dict(zip(grplbls,drggrps)).items(),2):
        if lbl1 == 'Low-dosage' or lbl2=='Low-dosage':
            continue
        rss,rspv = ranksums(DATA.loc[grp1].mean(axis=1),DATA.loc[grp2].mean(axis=1))
        tts,ttpv = ttest_ind(DATA.loc[grp1].mean(axis=1),DATA.loc[grp2].mean(axis=1))
        str_l=[lbl1,lbl2]
        ovrstats_d[(CL,'_'.join(str_l),'rs_stat')] = rss
        ovrstats_d[(CL,'_'.join(str_l),'rs_pval')] = rspv
        ovrstats_d[(CL,'_'.join(str_l),'tt_stat')] = tts
        ovrstats_d[(CL,'_'.join(str_l),'tt_pval')] = ttpv
        
    ax.set_title(CL,size=8)
    ax.set_yscale('log')
ax_l[0,0].set_ylabel('Viability',size=8)
plt.setp(ax_l[0,0].get_yticklabels(),size=6)

## make the synergy ratio Fig. S
dumdum = plt.setp(ax.get_xticklabels(),size=6,rotation=30,horizontalalignment='right')
axi = 0
for CL,rat_data in gross_synergy_stats.groupby(level=0):
    ax = ax_l[1,axi]
    #clr=clr_l[axi]
    axi+=1
    key_ovr = ['top', 'noncanc', 'anti', 'random']#list(set(rat_data.get_level_values(1)) & set(grplbl_d.keys()))
    clr_l = ['C3','C4','#EDB120','#4DBEEE','#E377c2']
    srat_data = rat_data.droplevel(0).loc[key_ovr,'ratio':'ratio_std']
    xvals = np.arange(srat_data.shape[0])
    for ii in range(srat_data.shape[0]):
        ax.errorbar(xvals[ii],srat_data.iloc[ii,0],yerr=srat_data.iloc[ii,1],fmt='.',
                   ecolor=clr_l[ii],markerfacecolor=clr_l[ii],markeredgecolor=clr_l[ii])
    ax.axhline(1,ls='--',color='C7')
    ax.set_xticks(xvals)
    ax.set_xticklabels([grplbl_d[kovr] for kovr in key_ovr],size=6,rotation=30)
    dumdum = plt.setp(ax.get_xticklabels(),size=6,rotation=30,horizontalalignment='right')
    #plt.setp(ax.get_yticklabels(),size=6)
    #ax.set_title(CL,size=8)
ax_l[1,0].set_ylabel('Synergy ratio',size=8)
plt.setp(ax_l[1,0].get_yticklabels(),size=6)
#ax.set_yscale('log',base=2)

fig.savefig('%s/fig4bc_gross_comparisons.svg'% OUTPUT_DIR)
## statistical comparisons of the boxplots
pd.Series(ovrstats_d).unstack([-2,-1]).to_excel('%s/fig4_viability_stats.xlsx'% OUTPUT_DIR)